In [ ]:
!pip install forex-python

In [ ]:
from forex_python.converter import CurrencyRates
c = CurrencyRates()
from tqdm import tqdm
import pandas as pd
from datetime import date, timedelta,datetime,date
import os


def get_week_number(date_obj):
  return (date_obj - date(date_obj.year,1,1)).days // 7 + 1

def get_weekday_number(date_obj):
  return (date_obj - date(date_obj.year,1,1)).days % 7


In [ ]:
from genericpath import samestat
import pdb
from tqdm import tqdm

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

def get_filename_from_YearAndWeek(YearAndWeek):
  year,week = YearAndWeek.split('_')
  return f'FOREX_Rate_{year}_week{week}.csv'


start_date = date(2000, 1, 1) #1st January 2000
end_date = date(2022, 12, 10) #10th December 2022

save_dir = 'FOREX_Rate_by_week'
os.makedirs(save_dir,exist_ok=True)

df_list = []
counter = 0

for single_date in tqdm(daterange(start_date, end_date)):
    counter +=1
    if counter % 100 == 0:
      print(f"{counter} days done")
   
    day_data = c.get_rates('USD', single_date)
    day_data['Date'] = single_date.strftime("%m/%d/%Y, %H:%M:%S")
    day_data['YearAndWeek'] = f'{single_date.year}_{get_week_number(single_date)}'
    df_list.append(day_data)

df = pd.DataFrame(df_list)
save_path = os.path.join(save_dir,get_filename_from_YearAndWeek(df_list[-1]['YearAndWeek']))
df.to_csv(save_path)

      

In [ ]:
!zip -r FOREX_Rate_by_week.zip FOREX_Rate_by_week/

In [ ]:
!du -h FOREX_Rate_by_week.zip

In [ ]:
!cp /content/FOREX_Rate_by_week.zip "/content/drive/MyDrive/BDA Forex prediction project/."

cp: cannot stat '/content/meta_data.zip': No such file or directory


In [ ]:
import datetime
def timestamp_to_week(timestamp):
  date = timestamp[:10]
  year,month,day = [int(val) for val in date.split("-")]
  week = datetime.date(year, month, day).isocalendar()[1]
  return str(week)

def get_year_weeks_start_and_end(year):
  start_dates = []
  end_dates = []

  first_day_of_year = date(year, 1, 1)
  week_start = first_day_of_year
  week_end = week_start+ timedelta(days=6)

  while week_end.year == year:
    start_dates.append(week_start.strftime("%m%d"))
    end_dates.append(week_end.strftime("%m%d"))

    week_start = week_end + timedelta(days=1)
    week_end = week_start+ timedelta(days=6)
  
  return start_dates,end_dates

In [ ]:

import requests
import math
import csv
import sys
import time
from tqdm import tqdm
import os
import pdb

# Data to be used while querying
url = 'https://api.nytimes.com/svc/search/v2/articlesearch.json?'
params = {
    "api-key": "k7d8PIJozXNxppfbVMcmqKXh1j2is4BQ",
    "fq": {
        "source": ["The New York Times"],
        "section_name": ["Your Money", "Job Market", "Business", "World", "Business Day", "Technology", "Financial", "Politics"],
        "document_type": ["article"]
    }
}
start_year = 20
end_year = 2023
api_keys = [
    "k7d8PIJozXNxppfbVMcmqKXh1j2is4BQ", 
    "srRafQT5F9h2QbSg2K7o5yvHyGGm6VXB",
    "ANAG6XCb39P2QjueyGhHh0DRo6YlgO3I", 
    "qUBjJKERphRkfn26FXuShub0zNSUM5qI", 
    "5IGEJJjATG4kG5jUR7ToPsNKqLNTK1xw", 
    "G38sQ8GrIeOAcRWrWK0ioJ5Sd4fuA3mV", 
    "W0J9rkchzD6YKgtAgm8qf8Unh42cgSWD",
    "0mXrz72SAxcLqdSubOZDMNTU40qflb6s"  
]

# Form the URL with the URL data
def generateURL(url, params):
    num_params = len(params.keys())
    for idx, param in enumerate(params):
        param_value = ""
        if param != 'fq':
            param_value = params[param]
        else:
            num_filters = len(params[param].keys())
            for index, filterField in enumerate(params[param]):
                filter_values = " ".join(['"{}"'.format(x) for x in params[param][filterField]])
                param_value += "{}:({})".format(filterField, filter_values)
                if index < num_filters-1:
                    param_value += ' AND '
        url += "{}={}".format(param, param_value)
        if idx < num_params-1:
            url += "&"
    return url

os.makedirs('meta_data',exist_ok=True)

for k in range(end_year - start_year):
    # For each year, find articles for each quarter
    week_start,week_end= get_year_weeks_start_and_end(start_year+k)
    for week in range(len(week_start)):
       
        begin_date = "{}{}".format(start_year+k, week_start[week])
        end_date = "{}{}".format(start_year+k, week_end[week])

        queryURL = generateURL(url,params)
        queryURL += "&begin_date={}&end_date={}".format(begin_date, end_date)

        # Loop through all pages (each page has data of 10 articles)
        api_swapper = 1
        current_api_key = api_keys[api_swapper]

        # Fet the data first to get the total number of hits
        print("{} - {}".format(start_year+k, week+1))
        try:
            response = (requests.get(queryURL)).json()
            # If some issue with response, switch the API key
            while True:
                if 'response' not in response:
                    api_swapper = (api_swapper + 1) % len(api_keys)
                    queryURL = queryURL.replace(current_api_key, api_keys[api_swapper])
                    current_api_key = api_keys[api_swapper]
                    time.sleep(0.8)
                    response = (requests.get(queryURL)).json()
                else:
                    break
            pages = min(1, math.ceil(response['response']['meta']['hits']/10))
            pbar = tqdm(total=pages) # Only to show the progress bar

            articles_meta_data = []
            file_count = 1
            
            # Keep track of duplicate articles
            duplicate_articles = []
            duplicate_articles_url = []
            
            with open('meta_data/articles_{}_week{}.csv'.format(start_year+k, week+1), 'w') as output_file:
                for i in range(pages):
                    current_url = queryURL + '&page={}'.format(i)
                    response = (requests.get(current_url)).json()
                    # If some issue with response, switch the API key
                    while True:
                        if 'response' not in response:
                            api_swapper = (api_swapper + 1) % len(api_keys)
                            queryURL = queryURL.replace(current_api_key, api_keys[api_swapper])
                            current_api_key = api_keys[api_swapper]
                            time.sleep(0.8)
                            response = (requests.get(queryURL)).json()
                        else:
                            break
                    articles = response['response']['docs']
                    for article in articles:
                        articles_meta_data.append({
                            "_id": article["_id"],
                            "url": article["web_url"], # URL of the article
                            "word_count": article["word_count"], # Number of words in the article
                            "section": article["section_name"], # News paper section under which the article is printed 
                            "date": article["pub_date"], # Article published date
                            "type": article["news_desk"], # Type of topic (business, sports, economic etc)
                            "headline": article["headline"]["main"].replace('\n', ' '), # Main headline of the article
                            "abstract": article["abstract"].replace('\n', ' ')
                        })
                    
                    # Update the progress bar
                    time.sleep(0.1)
                    pbar.update(1)

                    # Keep swapping API keys
                    api_swapper = (api_swapper + 1) % len(api_keys)
                    queryURL = queryURL.replace(current_api_key, api_keys[api_swapper])
                    current_api_key = api_keys[api_swapper]
                    time.sleep(0.8)
                keys = articles_meta_data[0].keys()
                writer = csv.DictWriter(output_file, keys)
                writer.writeheader()
                writer.writerows(articles_meta_data)
                pbar.close()
        except:
            print("Something went wrong while fetching the data! \n")
            print(sys.exc_info())
            print(response)
            raise

In [ ]:
import multiprocessing as mp
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv
import time
from tqdm import tqdm
import os
import os.path

# Total range of years
start_year = 2000
end_year = 2023

# Customise which year, quarter and row to start from and which year to end at
start_from_year = 2000
start_from_quarter = 1
start_from_row = 1
stop_at_year = 2023


def worker(row, q):
    row_text = []
    row['article'] = row['headline'] + row['abstract']
    row_text = [row['_id'],row['url'],row['word_count'],row['section'],row['date'],row['type'],row['headline'].replace('\n', ''),row['abstract'].replace('\n', ''),row['article'].replace('\n', '')]
    q.put(row_text)
    return row_text

def listener(q):
    while 1:
        m = q.get()
        if m == 'kill':
            break

for year in range(end_year - start_year):
    # Stop at the mentioned year
    if start_year+year == stop_at_year:
        break
    
    # Skip fetching previously covered data
    if (start_year+year < start_from_year):
        continue
    
    out_file_name = 'articles/articles_{}.csv'.format(start_year+year)
    columns = ['_id','url','word_count','section','date','type','headline','abstract','article']
    with open(out_file_name, 'a+') as out_file:
        writer = csv.writer(out_file)
        
        # If file is created new or empty, write column names first
        if os.path.isfile(out_file_name):
            if (os.stat(out_file_name).st_size == 0):
                writer.writerow(columns)
        else:
            writer.writerow(columns)

        print("processing year:",start_year+year)
        for week in tqdm(range(53)):
            
            # Skip Quarters as per the customisations
            if (start_year+year == start_from_year) and (week < start_from_quarter-1):
                continue
            if not os.path.exists('meta_data/articles_{}_week{}.csv'.format(start_year+year, week+1)):
                continue
            with open('meta_data/articles_{}_week{}.csv'.format(start_year+year, week+1)) as meta_file:
                data = csv.DictReader(meta_file)
                num_articles = 2000
                manager = mp.Manager()
                q = manager.Queue()    
                pool = mp.Pool(10)
                
                #put listener to work first
                watcher = pool.apply_async(listener, (q,))
                
                #fire off workers
                jobs = []
                for row in data:
                    job = pool.apply_async(worker, (row, q))
                    jobs.append(job)
                
                # collect results from the workers through the pool result queue
                for job in jobs:
                    row_val = job.get()
                    if not row_val:
                        continue
                    writer.writerow(row_val)
                    
                #now we are done, kill the listener
                q.put('kill')
                pool.close()
                pool.join()

In [ ]:
os.makedirs("/content/articles_by_week",exist_ok=True)

def parse_to_weeks(articles_folder,weeks_folder):
  for csv_file in os.listdir(articles_folder):
    csv_path = os.path.join(articles_folder,csv_file)
    df = pd.read_csv(csv_path)

    df['week'] = df['date'].apply(timestamp_to_week)

    unique_weeks = df.week.unique()

    for week in unique_weeks:
      week_path = os.path.join(weeks_folder,f"{csv_file.split('.')[0]}_week{week}.csv")
      df[df['week'] == week].to_csv(week_path)


parse_to_weeks("/content/articles","/content/articles_by_week")